Run in moalmanac directory, `/Users/brendan/moalmanac/moalmanac/` with the molecular oncology almanac virtual enviornment

In [1]:
import pandas as pd
import glob

handles = glob.glob('/Users/brendan/Github/moalmanac-paper/analyses/prospective-trial/formatted_variants/*')
handles = sorted(handles)

list_ = []
for handle in handles:
    variants = pd.read_csv(handle, sep='\t')
    list_.append(variants)
all_variants = pd.concat(list_, ignore_index=True)
all_variants['feature_type'].value_counts()

Somatic Variant             281
Copy Number                 195
Rearrangement                13
IHC                          12
Mutational Burden             5
Microsatellite Stability      1
Name: feature_type, dtype: int64

In [2]:
import subprocess

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from config import COLNAMES
from config import CONFIG

import datasources
import annotator
import evaluator
import features
import investigator
import matchmaker
import ontologymapper
import reporter
import writer

dbs = datasources.Datasources.generate_db_dict(CONFIG)
dbs['almanac_handle'] = 'datasources/moalmanac/moalmanac.json'

handle = '/Users/brendan/Github/moalmanac-paper/analyses/prospective-trial/paper/2019-Sicklick-supplement.xlsx'
patient_info = pd.read_excel(handle, sheet_name='Supplementary Table 1', header=2)
patient_info = patient_info.iloc[:83, :]
patient_info['Study ID'] = patient_info['Study ID'].astype(str)
patient_info.set_index('Study ID', inplace=True)

evaluated_germline = features.Features.create_empty_dataframe()
evaluated_ms_variants = features.Features.create_empty_dataframe()
evaluated_ms_status = features.Features.create_empty_dataframe()
evaluated_burden = features.Features.create_empty_dataframe()
evaluated_burden.loc[0, 'a'] = ''
evaluated_signatures = features.Features.create_empty_dataframe()
evaluated_wgd = features.Features.create_empty_dataframe()

handle = '/Users/brendan/Github/moalmanac-paper/analyses/prospective-trial/ontology.txt'
mapped_ontology = pd.read_csv(handle, sep='\t', index_col='Study ID')
mapped_ontology.index = mapped_ontology.index.astype(str)
mapped_ontology.fillna('')
mapped_ontology['code'] = mapped_ontology['code'].astype(str)
mapped_ontology['ontology'] = mapped_ontology['ontology'].astype(str)


def execute_cmd(command):
    subprocess.call(command, shell=True)

for handle in handles:
    cmd = 'rm almanac.additional.matches.json'
    execute_cmd(cmd)
    
    patient_id = handle.split('.formatted_variants.txt')[0].split('/')[-1]
    reported_tumor_type = patient_info.loc[patient_id, 'Diagnosis']
    code = mapped_ontology.loc[patient_id, 'code']
    ontology = mapped_ontology.loc[patient_id, 'ontology']
    
    patient = {
        'patient_id': patient_id,
        'reported_tumor_type': reported_tumor_type,
        'stage': 'NA',
        'description': '',
        'purity': 'NA',
        'ploidy': 'NA',
        'WGD': 'NA',
        'microsatellite_status': 'Unknown',
        'code': code,
        'ontology': ontology,
        'tumor_sample_barcode': '',
        'normal_sample_barcode': '',
        'oncotree_code': code,
        'oncotree_term': ontology
}
    
    variants = pd.read_csv(handle, sep='\t')
    variants = features.Features.preallocate_missing_columns(variants)
    
    somatic_variants = variants[variants['feature_type'].isin(['Somatic Variant', 'Copy Number', 'Rearrangement'])]
    annotated_somatic = annotator.Annotator.annotate_somatic_no_exac(somatic_variants, dbs, code)
    evaluated_somatic = evaluator.Evaluator.evaluate_somatic(annotated_somatic)
    
    if 'Mutational Burden' in variants['feature_type'].tolist():
        tmb_status = variants[variants['feature_type'].eq('Mutational Burden')].reset_index(drop=True)
        tmb_status = features.Features.preallocate_missing_columns(tmb_status)
        tmb_status[evaluator.Evaluator.high_burden_boolean] = True
        annotated_burden = annotator.Annotator.annotate_almanac(tmb_status, dbs, patient['code'])
        evaluated_burden = evaluator.Evaluator.evaluate_almanac(annotated_burden)
    else:
        evaluated_burden = features.Features.create_empty_dataframe()
        evaluated_burden.loc[0, 'a'] = ''
    
    if 'Microsatellite Stability' in variants['feature_type'].tolist():
        msi_status = variants[variants['feature_type'].eq('Microsatellite Stability')].reset_index(drop=True)
        msi_status = features.Features.preallocate_missing_columns(msi_status)
        annotated_ms_status = annotator.Annotator.annotate_almanac(msi_status, dbs, patient['code'])
        evaluated_ms_status = evaluator.Evaluator.evaluate_almanac(annotated_ms_status)
    else:
        evaluated_ms_status = features.Features.create_empty_dataframe()
    
    actionable = evaluator.Actionable.evaluate(evaluated_somatic, evaluated_germline,
                                               evaluated_ms_variants, evaluated_ms_status,
                                               evaluated_burden, evaluated_signatures, evaluated_wgd)
        
    dbs_preclinical = datasources.Preclinical.import_dbs()
    #efficacy_dictionary = investigator.SensitivityDictionary.create(dbs_preclinical, actionable, patient_id)
    #efficacy_summary = investigator.SummaryDataFrame.create(efficacy_dictionary, actionable, patient_id)
    #actionable = annotator.PreclinicalEfficacy.annotate(actionable, efficacy_summary)

    matchmaker_results = matchmaker.Matchmaker.compare(dbs, dbs_preclinical, evaluated_somatic, patient_id)
    
    report_dictionary = reporter.Reporter.generate_dictionary(evaluated_somatic, patient)
    report_dictionary['date'] = '17 Feb 2021'
    version_dictionary = reporter.Reporter.generate_version_dictionary()
    reporter.Reporter.generate_report(actionable, report_dictionary, version_dictionary,
                                      {}, pd.DataFrame(), 
                                      True, matchmaker_results, dbs_preclinical['dictionary'])
    
    writer.Actionable.write(actionable, patient_id)
    writer.PreclinicalMatchmaking.write(matchmaker_results, patient_id)
    
    ! mkdir -p "ipredict"
    ! mkdir -p "ipredict/"$patient_id
    ! mv $patient_id".actionable.txt" "ipredict/"$patient_id
    ! mv $patient_id".matchmaker.txt" "ipredict/"$patient_id
    ! mv build/index.html "ipredict/"$patient_id"/"$patient_id".report.html"

/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case.sort_values(by=SNFTypesCGCwithEvidence.label, ascending=True, inplace=True)
/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case.sort_values(by=SNFTypesCGCwithEvidence.label, ascending=True, inplace=True)
/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  somatic[cls.model_id] = cls.case_profile
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case.sort_values(by=SNFTypesCGCwithEvidence.label, ascending=True, inplace=True)
/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

/Users/brendan/Github/moalmanac/moalmanac/evaluator.py:307: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = cls.format_variant_classification(df[Evaluator.alt_type])
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  somatic[cls.model_id] = cls.case_profile
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

/Users/brendan/Github/moalmanac/moalmanac/annotator.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cls.score_bin] = cls.preallocate_bin(cls.score_bin, df.index)
/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/brendan/Github/moalmanac/moalmanac/annotator.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

...

/Users/brendan/Github/moalmanac/moalmanac/evaluator.py:307: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = cls.format_variant_classification(df[Evaluator.alt_type])
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  somatic[cls.model_id] = cls.case_profile
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  somatic[cls.model_id] = cls.case_profile
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case.sort_values(by=SNFTypesCGCwithEvidence.label, ascending=True, inplace=True)
/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

/Users/brendan/Github/moalmanac/moalmanac/annotator.py:1078: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cls.bin_name] = Annotator.match_ds(df, cls.create_msi_df(), cls.bin_name, cls.comparison_columns)
/Users/brendan/Github/moalmanac/moalmanac/evaluator.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cls.score_bin] = cls.map_almanac_bins(df[cls.almanac_bin])
/Users/brendan/Github/moalmanac/moalmanac/evaluator.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case.sort_values(by=SNFTypesCGCwithEvidence.label, ascending=True, inplace=True)
/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/brendan/Github/moalmanac/moalmanac/matchmaker.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case.sort_values(by=SNFTypesCGCwithEvidence.label, ascending=True, inplace=True)
/miniconda3/envs/moalmanac/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [3]:
list_ = []
handles = glob.glob('ipredict/*/*.actionable.txt')
for handle in handles:
    tmp = pd.read_csv(handle, sep='\t')
    list_.append(tmp)
    
all_actionable = pd.concat(list_, ignore_index=True)
all_actionable['feature_type'].value_counts()

Somatic Variant             184
Copy Number                  94
Rearrangement                 8
Mutational Burden             5
Microsatellite Stability      1
Name: feature_type, dtype: int64